In [2]:
'''
The below is a script for cleaning the housing market data after aggregation. Please see comments below 
on specific action taken. 
'''

import pandas as pd

# File path, loading the data, basic info and unique values
data_file = 'g:/My Drive/Portfolio/Projects/Complete/Allegheny County Housing/Datasets/full_housing_data.csv'
zip_city_file = 'g:/My Drive/Portfolio/Projects/Complete/Allegheny County Housing/Datasets/zip_city_check.csv'

df = pd.read_csv(data_file)

'''
# Inspect data
df.info()
df.nunique()
'''

# Remove duplicates
df = df.drop_duplicates()

# Removing "0 non-null" (MLS ID, Number of days on Zillow) and redundant values
columns_to_drop = ['MLS ID',
                   'Number of days on Zillow', 
                   'Living area unit', 
                   'Is owner occupied', 
                   'Property owned by Zillow'   
]

df.drop(columns=columns_to_drop, inplace=True)

# Rename column, as all units are sqft
df.rename(columns={'Living area': 'Living area - square footage'}, inplace=True)

# Convert 'Lot/land area' to acreage, drop unit column, and rename the column
sqft_to_acres = 1 / 43560
df['Lot/land area'] = df.apply(lambda row: row['Lot/land area'] * sqft_to_acres if row['Lot/land area unit'] == 'sqft' else row['Lot/land area'], axis=1)
df.drop(columns=['Lot/land area unit'], inplace=True)
df.rename(columns={'Lot/land area': 'Lot/land area (acreage)'}, inplace=True)

# Filter rows where State is 'PA'
df = df[df['State'] == 'PA']

# Convert 'Sold date (MM/DD/YYYY)' to DateTime
df['Sold date (MM/DD/YYYY)'] = pd.to_datetime(df['Sold date (MM/DD/YYYY)'], format='%m/%d/%Y')

# Assign columns to appropriate data types fill missing values
object_nulls = ['Property URL', 
                'Broker name',
                'Broker agent', 
                'Property unit number', 
                'Street address', 'City',
                'Listing Type'
]

number_nulls = ['Property price (USD)', 
                'Zestimate (USD)', 
                'Rent zestimate (USD per month)', 
                'Tax assessed value (USD)', 
                'Living area - square footage', 
                'Price per living area unit (USD)', 
                'Lot/land area (acreage)', 
                'Price per lot/land area unit (USD)', 
                'Bedrooms', 
                'Bathrooms', 
                'Zip', 
                'Latitude', 
                'Longitude'
]

df[object_nulls] = df[object_nulls].fillna('NULL')
df[number_nulls] = df[number_nulls].fillna(0)
df['Sold date (MM/DD/YYYY)'] = df['Sold date (MM/DD/YYYY)'].fillna(pd.to_datetime('1900-01-01', format='%Y-%m-%d'))

# Most of the columns ought to be ints rather than floats. Below is the conversion.
float_conversions = ['Zestimate (USD)', 
                     'Rent zestimate (USD per month)', 
                     'Tax assessed value (USD)', 
                     'Living area - square footage', 
                     'Bedrooms', 
                     'Bathrooms', 
                     'Zip'
]

df[float_conversions] = df[float_conversions].astype('int64')

'''
Here I noticed an issue with the "City" column where the names of cities weren't standardized. For example, 
East Pittsburgh and E Pittsburgh should both be under East Pittsburgh, some values show Bradford woods while 
others show Bradfordwoods, etc. 

I built a web scraper to standardize city names, pulling the zip code from each row and populating the 
recognized postal address, then used the resulting df to cross reference the proper city names.

You can find the separate script for the scraper on my GitHub. Below is further cleaning with the df from the scraper.
'''

# Load city_check_df
city_check_df = pd.read_csv(zip_city_file)

# Merge dataframes and update 'City' column
merged_df = df.merge(city_check_df, left_on='Zip', right_on='Zip Code', how='left')
merged_df['City'] = merged_df['City Name']
merged_df.drop(['Zip Code', 'City Name'], axis=1, inplace=True)
df = merged_df

# Drop rows with missing 'City', as they're not showing Allegheny County
df.dropna(subset=['City'], inplace=True)

'''
I also noticed both the "bedroom" and "bathroom" column both has 1 obscure value each, showing far
too many what there should be for the listing upon inspection. 
'''

# Find and remove the rows with the exorbitant values in the 'Bedrooms' and 'Bathrooms' columns
max_bedrooms_index = df['Bedrooms'].idxmax()
max_bathrooms_index = df['Bathrooms'].idxmax()
rows_to_remove = {max_bedrooms_index, max_bathrooms_index}
df = df.drop(rows_to_remove)

# Save the cleaned dataset
df.to_csv('g:/My Drive/Portfolio/Projects/Complete/Allegheny County Housing/Datasets/cleaned_housing_data.csv', index=False)

print("Cleaned")

Cleaned


In [3]:
df.head()

,Property URL,Property price (USD),Zestimate (USD),Rent zestimate (USD per month),Tax assessed value (USD),Sold date (MM/DD/YYYY),Living area - square footage,Price per living area unit (USD),Lot/land area (acreage),Price per lot/land area unit (USD),...,Broker agent,Property unit number,Street address,City,Zip,State,Country,Latitude,Longitude,Listing Type
0,https://www.zillow.com/homedetails/1316-Wesley...,14900,0,1530,36000,2023-10-16,2830,5.27,0.0723,4.73,...,Kathy Armstrong,NULL,1316 Wesley St,Pittsburgh,15221,PA,USA,40.443203,-79.872240,houses
1,https://www.zillow.com/homedetails/443-Porter-...,13900,0,1300,79600,2023-10-13,1264,11.00,0.1300,2.45,...,Robert Moncavage,NULL,443 Porter St,North Versailles,15137,PA,USA,40.387325,-79.835205,houses
2,https://www.zillow.com/homedetails/412-W-9th-A...,15000,0,1214,50700,2023-10-10,1712,8.76,0.1377,2.50,...,Mary Ann Urbanski,NULL,412 W 9th Ave,Tarentum,15084,PA,USA,40.599342,-79.766690,houses
3,https://www.zillow.com/homedetails/Shafton-Ave...,5800,0,1376,1110,2023-10-10,0,0.00,0.2900,19739.67,...,NULL,NULL,Shafton Ave,Irwin,15642,PA,USA,40.349600,-79.782950,houses
4,https://www.zillow.com/homedetails/214-Grape-W...,12000,0,849,12800,2023-09-26,825,14.55,0.0505,5.46,...,Gerald Irwin,NULL,214 Grape Way,Clairton,15025,PA,USA,40.306038,-79.886505,houses
